# Download Automatizado de Sentenças - TJMA
## Extração completa de sentenças judiciais via Selenium

Este notebook realiza download automatizado de sentenças do Tribunal de Justiça do Maranhão (TJMA).

**Autor:** Mestrando em Administração Pública - IDP  
**Propósito:** Pesquisa acadêmica com dados públicos  
**Versão:** 3.0 - Totalmente funcional  
**Data:** Dezembro/2024

---

## 📋 Fluxo Completo:

1. Acessa consulta pública do PJe-TJMA
2. Busca processo pelo número
3. Acessa página de detalhes
4. Identifica documentos (sentenças, decisões, etc.)
5. Baixa os PDFs
6. Salva metadados em CSV

## 1. Instalação de Dependências

In [1]:
#!pip install selenium webdriver-manager pandas beautifulsoup4 lxml requests

   ---------------------------------------- 0.0/9.7 MB ? eta -:--:--
   -- ------------------------------------- 0.5/9.7 MB 5.4 MB/s eta 0:00:02
   ------- -------------------------------- 1.8/9.7 MB 6.4 MB/s eta 0:00:02
   ------------ --------------------------- 3.1/9.7 MB 6.4 MB/s eta 0:00:02
   ----------------- ---------------------- 4.2/9.7 MB 6.9 MB/s eta 0:00:01
   ----------------- ---------------------- 4.2/9.7 MB 6.9 MB/s eta 0:00:01
   ----------------------- ---------------- 5.8/9.7 MB 5.0 MB/s eta 0:00:01
   -------------------------------- ------- 7.9/9.7 MB 5.9 MB/s eta 0:00:01
   ---------------------------------------- 9.7/9.7 MB 6.4 MB/s eta 0:00:00

  Attempting uninstall: urllib3

    Found existing installation: urllib3 2.3.0

    Uninstalling urllib3-2.3.0:

      Successfully uninstalled urllib3-2.3.0

   ---- ----------------------------------- 1/9 [urllib3]
  Attempting uninstall: typing_extensions
   ---- ----------------------------------- 1/9 [urllib3]
   -

## 2. Importações

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd
import time
import os
import re
import json
import requests
from datetime import datetime
from urllib.parse import urljoin, urlparse, parse_qs

print("✓ Bibliotecas importadas com sucesso!")

✓ Bibliotecas importadas com sucesso!


## 3. Configurações

In [2]:
# Diretórios de saída
OUTPUT_DIR = "sentencas_tjma"
PDF_DIR = os.path.join(OUTPUT_DIR, "pdfs")
HTML_DIR = os.path.join(OUTPUT_DIR, "html")
METADATA_DIR = os.path.join(OUTPUT_DIR, "metadados")

# Criar diretórios
for dir_path in [OUTPUT_DIR, PDF_DIR, HTML_DIR, METADATA_DIR]:
    os.makedirs(dir_path, exist_ok=True)

# URLs
URL_CONSULTA = "https://pje.tjma.jus.br/pje/ConsultaPublica/listView.seam"
URL_BASE = "https://pje.tjma.jus.br"

print(f"✓ Estrutura de diretórios criada:")
print(f"  - PDFs: {PDF_DIR}")
print(f"  - HTML: {HTML_DIR}")
print(f"  - Metadados: {METADATA_DIR}")

✓ Estrutura de diretórios criada:
  - PDFs: sentencas_tjma\pdfs
  - HTML: sentencas_tjma\html
  - Metadados: sentencas_tjma\metadados


## 4. Funções Auxiliares

In [3]:
def configurar_driver(headless=False, download_dir=None):
    """
    Configura o Chrome WebDriver
    
    Args:
        headless (bool): Executar sem interface gráfica
        download_dir (str): Diretório para downloads
    
    Returns:
        webdriver: Chrome WebDriver configurado
    """
    chrome_options = Options()
    
    if headless:
        chrome_options.add_argument('--headless')
    
    # Configurações de performance e compatibilidade
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    chrome_options.add_argument('--disable-blink-features=AutomationControlled')
    chrome_options.add_experimental_option('excludeSwitches', ['enable-automation'])
    chrome_options.add_experimental_option('useAutomationExtension', False)
    
    # User agent
    chrome_options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36')
    
    # Configurar diretório de download
    if download_dir:
        prefs = {
            'download.default_directory': download_dir,
            'download.prompt_for_download': False,
            'download.directory_upgrade': True,
            'plugins.always_open_pdf_externally': True
        }
        chrome_options.add_experimental_option('prefs', prefs)
    
    # Inicializar driver
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=chrome_options)
    
    # Timeouts
    driver.set_page_load_timeout(60)
    driver.implicitly_wait(10)
    
    return driver


def limpar_nome_arquivo(texto):
    """
    Remove caracteres inválidos de nomes de arquivo
    """
    return re.sub(r'[<>:"/\\|?*]', '_', texto)


def extrair_numero_processo_limpo(numero):
    """
    Remove formatação do número do processo
    Ex: 0800151-71.2021.8.10.0056 -> 08001517120218100056
    """
    return re.sub(r'[^0-9]', '', numero)


print("✓ Funções auxiliares definidas")

✓ Funções auxiliares definidas


## 5. Função Principal - Buscar Processo

In [4]:
def buscar_processo(driver, numero_processo):
    """
    Busca um processo no PJe-TJMA e retorna os detalhes
    
    Args:
        driver: WebDriver do Selenium
        numero_processo (str): Número do processo (formato CNJ)
    
    Returns:
        dict: Dados do processo ou None
    """
    try:
        print(f"\n{'='*70}")
        print(f"🔍 Buscando processo: {numero_processo}")
        print(f"{'='*70}")
        
        # Acessar página de consulta
        print("[1/5] Acessando consulta pública...")
        driver.get(URL_CONSULTA)
        time.sleep(3)
        
        # Localizar e preencher campo
        print("[2/5] Preenchendo número do processo...")
        campo_processo = WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.ID, "fPP:numProcesso-inputNumeroProcessoDecoration:numProcesso-inputNumeroProcesso"))
        )
        campo_processo.clear()
        campo_processo.send_keys(numero_processo)
        time.sleep(2)
        
        # Clicar em pesquisar
        print("[3/5] Enviando pesquisa...")
        botao_pesquisar = driver.find_element(By.ID, "fPP:searchProcessos")
        botao_pesquisar.click()
        time.sleep(5)  # Aguardar AJAX
        
        # Verificar resultados
        print("[4/5] Verificando resultados...")
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        
        # Procurar pela tabela de resultados
        tabela = soup.find('tbody', id='fPP:processosTable:tb')
        
        if not tabela or not tabela.find('tr'):
            print("\n❌ Processo não encontrado!")
            return None
        
        # Extrair link de detalhes
        link_detalhes = tabela.find('a', href=True)
        if not link_detalhes:
            print("\n❌ Link de detalhes não encontrado!")
            return None
        
        # Extrair URL do popup
        onclick = link_detalhes.get('onclick', '')
        match = re.search(r"openPopUp\([^,]+,\s*'([^']+)'\)", onclick)
        
        if not match:
            print("\n❌ URL de detalhes não encontrada!")
            return None
        
        url_detalhes = match.group(1)
        url_completa = urljoin(URL_BASE, url_detalhes)
        
        print(f"[5/5] Processo encontrado!")
        print(f"✓ URL de detalhes: {url_completa[:80]}...")
        
        return {
            'numero': numero_processo,
            'url_detalhes': url_completa,
            'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        }
        
    except Exception as e:
        print(f"\n❌ ERRO: {str(e)}")
        return None


print("✓ Função de busca definida")

✓ Função de busca definida


## 6. Função - Extrair Documentos

In [5]:
# FUNÇÃO MELHORADA - Cole no lugar da função extrair_documentos
# Corrige o problema do "Visualizar documentos" aparecendo no texto

def extrair_documentos(driver, url_detalhes, numero_processo):
    """
    Extrai lista de documentos (sentenças, decisões, etc.) do processo
    
    Args:
        driver: WebDriver do Selenium
        url_detalhes (str): URL da página de detalhes
        numero_processo (str): Número do processo
    
    Returns:
        list: Lista de dicionários com dados dos documentos
    """
    try:
        print(f"\n{'='*70}")
        print(f"📄 Extraindo documentos do processo...")
        print(f"{'='*70}")
        
        # Acessar página de detalhes
        print("[1/3] Acessando detalhes do processo...")
        driver.get(url_detalhes)
        time.sleep(5)
        
        # Salvar HTML para análise
        numero_limpo = extrair_numero_processo_limpo(numero_processo)
        html_file = os.path.join(HTML_DIR, f"{numero_limpo}_detalhes.html")
        with open(html_file, 'w', encoding='utf-8') as f:
            f.write(driver.page_source)
        print(f"✓ HTML salvo: {html_file}")
        
        # Parse HTML
        print("[2/3] Analisando documentos...")
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        
        # Procurar tabela pelo ID que contém 'processoDocumentoGridTab'
        tabela = None
        todas_tabelas = soup.find_all('table')
        
        for t in todas_tabelas:
            table_id = t.get('id', '')
            if 'processoDocumentoGridTab' in table_id and ':tb' not in table_id:
                tabela = t
                print(f"✓ Tabela encontrada: {table_id}")
                break
        
        if not tabela:
            print("\n❌ Tabela de documentos não encontrada!")
            return []
        
        # Encontrar tbody
        tbody = tabela.find('tbody')
        
        if not tbody:
            tbody = soup.find('tbody', id=lambda x: x and 'processoDocumentoGridTab:tb' in x)
        
        if not tbody:
            print("\n❌ tbody não encontrado!")
            return []
        
        tbody_id = tbody.get('id', 'SEM ID')
        print(f"✓ tbody encontrado: {tbody_id}")
        
        # Encontrar linhas
        linhas = tbody.find_all('tr', class_='rich-table-row')
        
        print(f"[3/3] Encontradas {len(linhas)} linhas na tabela...")
        
        documentos = []
        
        for idx, linha in enumerate(linhas):
            try:
                # Procurar link com onclick que contém 'documentoSemLoginHTML'
                links = linha.find_all('a')
                
                link_documento = None
                for link in links:
                    onclick = link.get('onclick', '')
                    if 'documentoSemLoginHTML' in onclick:
                        link_documento = link
                        break
                
                if not link_documento:
                    continue
                
                # Extrair texto (data/hora e tipo)
                texto = link_documento.get_text(strip=True)
                
                # CORREÇÃO: Remover textos acessórios como "Visualizar documentos"
                texto = texto.replace('Visualizar documentos', '').strip()
                texto = texto.replace('Visualizar', '').strip()
                
                # Extrair URL do popup
                onclick = link_documento.get('onclick', '')
                match = re.search(r"openPopUp\([^,]+,\s*'([^']+)'\)", onclick)
                
                if not match:
                    continue
                
                url_doc = match.group(1)
                url_doc = url_doc.replace('&amp;', '&')
                
                # Garantir URL completa
                if not url_doc.startswith('http'):
                    url_completa = urljoin(URL_BASE, url_doc)
                else:
                    url_completa = url_doc
                
                # Extrair data/hora e tipo
                partes = texto.split(' - ', 1)
                data_hora = partes[0].strip() if len(partes) > 0 else 'N/A'
                tipo_doc = partes[1].strip() if len(partes) > 1 else texto.strip()
                
                # Se não conseguiu separar, tenta outro formato
                if tipo_doc == 'N/A' or not tipo_doc:
                    tipo_doc = texto.strip()
                
                # Verificar se é sentença ou decisão
                tipo_lower = tipo_doc.lower()
                eh_sentenca = any(palavra in tipo_lower for palavra in ['sentença', 'sentenÃ§a', 'sentenca', 'ata de audiência'])
                eh_decisao = any(palavra in tipo_lower for palavra in ['decisão', 'decisÃ£o', 'decisao'])
                
                documento = {
                    'id': idx,
                    'data_hora': data_hora,
                    'tipo': tipo_doc,
                    'url': url_completa,
                    'eh_sentenca': eh_sentenca,
                    'eh_decisao': eh_decisao
                }
                
                documentos.append(documento)
                
                tipo_badge = "🏛️ SENTENÇA" if eh_sentenca else ("⚖️  DECISÃO" if eh_decisao else "📄 DOCUMENTO")
                print(f"  {tipo_badge} - {data_hora} - {tipo_doc[:50]}")
                
            except Exception as e:
                print(f"  ⚠️  Erro ao processar linha {idx}: {str(e)}")
                continue
        
        print(f"\n✅ Total de documentos encontrados: {len(documentos)}")
        
        if len(documentos) > 0:
            sentencas = sum(1 for d in documentos if d['eh_sentenca'])
            decisoes = sum(1 for d in documentos if d['eh_decisao'])
            print(f"   - Sentenças: {sentencas}")
            print(f"   - Decisões: {decisoes}")
            print(f"   - Outros: {len(documentos) - sentencas - decisoes}")
        
        return documentos
        
    except Exception as e:
        print(f"\n❌ ERRO ao extrair documentos: {str(e)}")
        import traceback
        traceback.print_exc()
        return []


print("✓ Função de extração de documentos definida (VERSÃO MELHORADA)")

✓ Função de extração de documentos definida (VERSÃO MELHORADA)


## 7. Função - Baixar Documento

In [6]:
def baixar_documento(driver, documento, numero_processo):
    """
    Baixa um documento (sentença/decisão) do processo
    
    Args:
        driver: WebDriver do Selenium
        documento (dict): Dados do documento
        numero_processo (str): Número do processo
    
    Returns:
        str: Caminho do arquivo salvo ou None
    """
    try:
        numero_limpo = extrair_numero_processo_limpo(numero_processo)
        
        # CORREÇÃO: Simplificar nome do arquivo
        # Usar apenas data e tipo principal
        data_limpa = documento['data_hora'].replace('/', '-').replace(':', '-').replace(' ', '_')
        
        # Extrair tipo principal (antes do parêntese)
        tipo_principal = documento['tipo'].split('(')[0].strip()
        tipo_limpo = limpar_nome_arquivo(tipo_principal)
        
        # Limitar tamanho do tipo para evitar nomes muito longos
        if len(tipo_limpo) > 50:
            tipo_limpo = tipo_limpo[:50]
        
        # ID único baseado no índice
        doc_id = documento['id']
        
        # Nome do arquivo simplificado
        nome_arquivo = f"{numero_limpo}_{doc_id:03d}_{data_limpa}_{tipo_limpo}.html"
        caminho_completo = os.path.join(PDF_DIR, nome_arquivo)
        
        print(f"\n⬇️  Baixando: {tipo_principal[:50]}...")
        print(f"   Arquivo: {nome_arquivo}")
        
        # Acessar URL do documento
        driver.get(documento['url'])
        time.sleep(3)
        
        # Salvar conteúdo
        with open(caminho_completo, 'w', encoding='utf-8') as f:
            f.write(driver.page_source)
        
        print(f"   ✅ Salvo: {caminho_completo}")
        
        return caminho_completo
        
    except Exception as e:
        print(f"   ❌ Erro ao baixar: {str(e)}")
        
        # Tentativa alternativa com nome ainda mais curto
        try:
            nome_curto = f"{numero_limpo}_{documento['id']:03d}.html"
            caminho_alternativo = os.path.join(PDF_DIR, nome_curto)
            
            with open(caminho_alternativo, 'w', encoding='utf-8') as f:
                f.write(driver.page_source)
            
            print(f"   ✅ Salvo com nome alternativo: {caminho_alternativo}")
            return caminho_alternativo
            
        except Exception as e2:
            print(f"   ❌ Falha na tentativa alternativa: {str(e2)}")
            return None


print("✓ Função de download definida (VERSÃO CORRIGIDA)")

✓ Função de download definida (VERSÃO CORRIGIDA)


## 8. Função Completa - Pipeline

In [7]:
def processar_processo_completo(numero_processo, headless=False, apenas_sentencas=True):
    """
    Pipeline completo: busca, extrai e baixa documentos de um processo
    
    Args:
        numero_processo (str): Número do processo
        headless (bool): Executar sem interface gráfica
        apenas_sentencas (bool): Baixar apenas sentenças (True) ou todos documentos (False)
    
    Returns:
        dict: Resultado do processamento
    """
    driver = None
    
    try:
        print(f"\n{'#'*70}")
        print(f"# PROCESSANDO PROCESSO: {numero_processo}")
        print(f"{'#'*70}")
        
        # Configurar driver
        print("\n⚙️  Configurando navegador...")
        driver = configurar_driver(headless=headless, download_dir=PDF_DIR)
        
        # Buscar processo
        dados_processo = buscar_processo(driver, numero_processo)
        
        if not dados_processo:
            return {
                'sucesso': False,
                'erro': 'Processo não encontrado'
            }
        
        # Extrair documentos
        documentos = extrair_documentos(driver, dados_processo['url_detalhes'], numero_processo)
        
        if not documentos:
            return {
                'sucesso': False,
                'erro': 'Nenhum documento encontrado'
            }
        
        # Filtrar documentos
        if apenas_sentencas:
            docs_para_baixar = [d for d in documentos if d['eh_sentenca']]
            print(f"\n🎯 Baixando apenas sentenças: {len(docs_para_baixar)} documento(s)")
        else:
            docs_para_baixar = documentos
            print(f"\n📥 Baixando todos documentos: {len(docs_para_baixar)} documento(s)")
        
        # Baixar documentos
        arquivos_baixados = []
        
        for doc in docs_para_baixar:
            caminho = baixar_documento(driver, doc, numero_processo)
            if caminho:
                doc['arquivo_local'] = caminho
                arquivos_baixados.append(caminho)
            time.sleep(2)  # Intervalo entre downloads
        
        # Salvar metadados
        numero_limpo = extrair_numero_processo_limpo(numero_processo)
        
        # CSV
        csv_file = os.path.join(METADATA_DIR, f"{numero_limpo}_metadados.csv")
        df = pd.DataFrame(documentos)
        df.to_csv(csv_file, index=False, encoding='utf-8-sig')
        print(f"\n💾 Metadados salvos: {csv_file}")
        
        # JSON
        json_file = os.path.join(METADATA_DIR, f"{numero_limpo}_completo.json")
        resultado = {
            'processo': dados_processo,
            'documentos': documentos,
            'arquivos_baixados': arquivos_baixados,
            'total_documentos': len(documentos),
            'total_baixados': len(arquivos_baixados)
        }
        
        with open(json_file, 'w', encoding='utf-8') as f:
            json.dump(resultado, f, ensure_ascii=False, indent=2)
        print(f"💾 JSON completo salvo: {json_file}")
        
        # Resumo
        print(f"\n{'='*70}")
        print(f"✅ PROCESSAMENTO CONCLUÍDO COM SUCESSO!")
        print(f"{'='*70}")
        print(f"📊 Resumo:")
        print(f"   - Processo: {numero_processo}")
        print(f"   - Documentos encontrados: {len(documentos)}")
        print(f"   - Arquivos baixados: {len(arquivos_baixados)}")
        print(f"   - Diretório: {PDF_DIR}")
        
        resultado['sucesso'] = True
        return resultado
        
    except Exception as e:
        print(f"\n❌ ERRO CRÍTICO: {str(e)}")
        import traceback
        traceback.print_exc()
        return {
            'sucesso': False,
            'erro': str(e)
        }
        
    finally:
        if driver:
            print("\n🔚 Fechando navegador...")
            driver.quit()


print("✓ Pipeline completo definido")

✓ Pipeline completo definido


## 9. EXECUTAR - Processar Um Processo

In [8]:
# ============================================================
# CONFIGURAÇÃO
# ============================================================

# Número do processo (formato CNJ)
NUMERO_PROCESSO = "0800151-71.2021.8.10.0056"

# Configurações
HEADLESS = False  # False = Mostra navegador (útil para debug)
APENAS_SENTENCAS = True  # True = Baixa só sentenças, False = Baixa tudo

# ============================================================
# EXECUTAR
# ============================================================

resultado = processar_processo_completo(
    numero_processo=NUMERO_PROCESSO,
    headless=HEADLESS,
    apenas_sentencas=APENAS_SENTENCAS
)

# Exibir resultado
print("\n" + "="*70)
if resultado['sucesso']:
    print("✅ Processo finalizado com sucesso!")
    print(f"\n📁 Verifique os arquivos em:")
    print(f"   - PDFs: {PDF_DIR}")
    print(f"   - Metadados: {METADATA_DIR}")
else:
    print(f"❌ Erro: {resultado.get('erro', 'Desconhecido')}")


######################################################################
# PROCESSANDO PROCESSO: 0800151-71.2021.8.10.0056
######################################################################

⚙️  Configurando navegador...

🔍 Buscando processo: 0800151-71.2021.8.10.0056
[1/5] Acessando consulta pública...
[2/5] Preenchendo número do processo...
[3/5] Enviando pesquisa...
[4/5] Verificando resultados...
[5/5] Processo encontrado!
✓ URL de detalhes: https://pje.tjma.jus.br/pje/ConsultaPublica/DetalheProcessoConsultaPublica/listV...

📄 Extraindo documentos do processo...
[1/3] Acessando detalhes do processo...
✓ HTML salvo: sentencas_tjma\html\08001517120218100056_detalhes.html
[2/3] Analisando documentos...
✓ Tabela encontrada: j_id143:processoDocumentoGridTab
✓ tbody encontrado: j_id143:processoDocumentoGridTab:tb
[3/3] Encontradas 15 linhas na tabela...
  🏛️ SENTENÇA - 22/09/2025 16:18:06 - Sentença (Sentença)
  🏛️ SENTENÇA - 03/09/2025 09:08:28 - Ata de audiência com despacho, decisã

## 10. EXECUTAR - Processar Múltiplos Processos

In [9]:
# Lista de processos para processar
LISTA_PROCESSOS = [
    "0800151-71.2021.8.10.0056",
    "0803788-59.2023.8.10.0056", 
    "0803789-44.2023.8.10.0056", 
    "0877326-44.2023.8.10.0001", 
    "0800643-24.2025.8.10.0056", 
    "0002347-86.2017.8.10.0056", 
    "0804518-70.2023.8.10.0056", 
    "0803042-60.2024.8.10.0056", 
    "0805337-70.2024.8.10.0056"
]

# Configurações
HEADLESS = True  # Recomendado True para múltiplos processos
APENAS_SENTENCAS = True
INTERVALO_ENTRE_PROCESSOS = 10  # segundos

# Processar todos
resultados = []

for idx, numero in enumerate(LISTA_PROCESSOS, 1):
    print(f"\n\n{'#'*70}")
    print(f"# PROCESSO {idx}/{len(LISTA_PROCESSOS)}")
    print(f"{'#'*70}")
    
    resultado = processar_processo_completo(
        numero_processo=numero,
        headless=HEADLESS,
        apenas_sentencas=APENAS_SENTENCAS
    )
    
    resultados.append({
        'processo': numero,
        'sucesso': resultado['sucesso'],
        'erro': resultado.get('erro'),
        'total_baixados': resultado.get('total_baixados', 0)
    })
    
    # Intervalo entre processos
    if idx < len(LISTA_PROCESSOS):
        print(f"\n⏳ Aguardando {INTERVALO_ENTRE_PROCESSOS}s antes do próximo processo...")
        time.sleep(INTERVALO_ENTRE_PROCESSOS)

# Relatório final
print(f"\n\n{'#'*70}")
print(f"# RELATÓRIO FINAL")
print(f"{'#'*70}")

df_resultados = pd.DataFrame(resultados)
print(df_resultados.to_string(index=False))

# Salvar relatório
relatorio_file = os.path.join(METADATA_DIR, f"relatorio_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv")
df_resultados.to_csv(relatorio_file, index=False, encoding='utf-8-sig')
print(f"\n📊 Relatório salvo: {relatorio_file}")

# Estatísticas
total = len(resultados)
sucessos = sum(1 for r in resultados if r['sucesso'])
falhas = total - sucessos
total_docs = sum(r['total_baixados'] for r in resultados)

print(f"\n📈 Estatísticas:")
print(f"   - Total de processos: {total}")
print(f"   - Sucessos: {sucessos}")
print(f"   - Falhas: {falhas}")
print(f"   - Total de documentos baixados: {total_docs}")



######################################################################
# PROCESSO 1/9
######################################################################

######################################################################
# PROCESSANDO PROCESSO: 0800151-71.2021.8.10.0056
######################################################################

⚙️  Configurando navegador...

🔍 Buscando processo: 0800151-71.2021.8.10.0056
[1/5] Acessando consulta pública...
[2/5] Preenchendo número do processo...
[3/5] Enviando pesquisa...
[4/5] Verificando resultados...
[5/5] Processo encontrado!
✓ URL de detalhes: https://pje.tjma.jus.br/pje/ConsultaPublica/DetalheProcessoConsultaPublica/listV...

📄 Extraindo documentos do processo...
[1/3] Acessando detalhes do processo...
✓ HTML salvo: sentencas_tjma\html\08001517120218100056_detalhes.html
[2/3] Analisando documentos...
✓ Tabela encontrada: j_id143:processoDocumentoGridTab
✓ tbody encontrado: j_id143:processoDocumentoGridTab:tb
[3/3] Encontrad

## 11. Carregar Processos de um CSV

In [12]:
# Se você tem um CSV com números de processos
# Formato esperado: coluna chamada 'numero_processo' ou 'processo'

CSV_PROCESSOS = "processos-tjma-50.csv"  # <-- AJUSTE O CAMINHO

# Ler CSV
try:
    df_processos = pd.read_csv(CSV_PROCESSOS)
    
    # Identificar coluna com números
    coluna_numero = None
    for col in ['numero_processo', 'processo', 'numeroProcesso', 'num_processo']:
        if col in df_processos.columns:
            coluna_numero = col
            break
    
    if not coluna_numero:
        print("❌ Coluna de número de processo não encontrada!")
        print(f"   Colunas disponíveis: {df_processos.columns.tolist()}")
    else:
        LISTA_PROCESSOS = df_processos[coluna_numero].dropna().astype(str).tolist()
        print(f"✅ {len(LISTA_PROCESSOS)} processos carregados do CSV")
        print(f"   Primeiros 5: {LISTA_PROCESSOS[:5]}")
        
        # Agora execute a célula anterior (10) para processar todos
        
except FileNotFoundError:
    print(f"❌ Arquivo não encontrado: {CSV_PROCESSOS}")
except Exception as e:
    print(f"❌ Erro ao ler CSV: {str(e)}")

✅ 50 processos carregados do CSV
   Primeiros 5: ['0800424-45.2024.8.10.0056', '0802205-39.2023.8.10.0056', '0801845-70.2024.8.10.0056', '0802176-52.2024.8.10.0056', '0805298-73.2024.8.10.0056']


---

## 📚 Documentação para o Paper

### Metodologia de Coleta

**Fonte de Dados:**  
Tribunal de Justiça do Maranhão (TJMA) - Sistema PJe (Processo Judicial Eletrônico)

**Técnica:**  
Web scraping automatizado utilizando Selenium WebDriver para simular navegação e extração de documentos públicos

**Estrutura do Processo:**
1. Consulta pública via PJe
2. Identificação de processos por número CNJ
3. Extração de metadados e documentos
4. Download de sentenças e decisões

**Conformidade Legal:**
- LGPD: Dados públicos judiciais
- Resolução nº 121/2010 CNJ: Publicidade processual
- Acesso via consulta pública oficial

**Citação Sugerida (ABNT):**

TRIBUNAL DE JUSTIÇA DO MARANHÃO. **Sistema PJe - Consulta Pública**. São Luís, 2024. Disponível em: https://pje.tjma.jus.br/pje/ConsultaPublica/listView.seam. Acesso em: [data da consulta].

CONSELHO NACIONAL DE JUSTIÇA. **Justiça em Números 2024**. Brasília: CNJ, 2024. Disponível em: https://www.cnj.jus.br/pesquisas-judiciarias/justica-em-numeros/. Acesso em: [data].

### Limitações

- Apenas processos públicos (não incluem processos em segredo de justiça)
- Dependência de disponibilidade do sistema PJe
- Documentos em formato HTML (conversão para texto pode ser necessária)

### Estrutura de Dados Gerados

**Metadados (CSV/JSON):**
- Número do processo
- Data e hora dos documentos
- Tipo de documento
- URLs de acesso
- Classificação (sentença/decisão/outro)

**Arquivos:**
- HTML dos documentos
- Metadados estruturados
- Logs de processamento